# AoC 2024 Day 14
https://adventofcode.com/2024/day/14

In [54]:
import time
from IPython.display import clear_output
import re
from collections import Counter

In [55]:
with open('data/day14.txt') as f:
    data = f.read().split('\n')
data

# WIDTH, HEIGHT = 11, 7  # test
WIDTH, HEIGHT = 101, 103  # real

In [56]:
position = tuple[int, int]
velocity = tuple[int, int]

def parse_robot_data(data) -> list[tuple[position,velocity]]:
    result = []
    pattern = re.compile(r'p\=(-?\d+),(-?\d+) v=(-?\d+),(-?\d+)')
    for row in data:
        row_match = pattern.search(row)
        result.append(((int(row_match[1]), int(row_match[2])), (int(row_match[3]), int(row_match[4]))))
    return result

parsed_data = parse_robot_data(data)
parsed_data[:10]

[((1, 65), (-5, -84)),
 ((14, 63), (-85, 6)),
 ((49, 81), (71, 14)),
 ((61, 77), (85, 67)),
 ((58, 37), (42, 47)),
 ((99, 13), (-4, -72)),
 ((50, 29), (58, 96)),
 ((84, 63), (-93, -25)),
 ((77, 90), (-3, -70)),
 ((11, 82), (-74, 54))]

In [79]:
def print_positions(positions: list[position], show_quadrants=False) -> None:
    positions = Counter(positions)
    builder = []
    for h in range(HEIGHT):
        builder.append([])
        for w in range(WIDTH):
            if show_quadrants and (h == HEIGHT // 2 or w == WIDTH // 2):
                builder[-1].append(' ')
            elif (w,h) in positions:
                builder[-1].append(str(positions[(w,h)]))
            else:
                builder[-1].append('.')
    return '\n'.join([''.join(row) for row in builder])
print(print_positions([x[0] for x in parsed_data]))

.................................1..........................................................1..1.....
......................................1............................................................1.
................1..................1.1...............................................................
.........1.........................1........1...........................................1.....1......
................................1....................................................................
....1.....................................................1..........................................
.......1.........1...........................1................1...........1..........................
...................1....1.............1..........1.....1.............................................
......................................................1..............................................
...................................1..1..2....1............................1......

In [80]:
def find_position_after_seconds(p: position, v: velocity, t: int) -> position:
    return ((p[0]+v[0]*t)%WIDTH, (p[1]+v[1]*t)%HEIGHT)

def find_positions(parsed_data, t) -> list[position]:
    return [find_position_after_seconds(*row, t) for row in parsed_data]

def find_safety_factor(positions: list[position]) -> int:
    q1,q2,q3,q4 = 0,0,0,0
    for px,py in positions:
        if px < WIDTH //2 and py < HEIGHT//2:
            q1 += 1
        elif px < WIDTH //2 and py > HEIGHT//2:
            q2 += 1
        elif px > WIDTH //2 and py < HEIGHT//2:
            q3 += 1
        elif px > WIDTH //2 and py > HEIGHT//2:
            q4 += 1
    return q1*q2*q3*q4
        
pos = find_positions(parsed_data, 100)
print_positions(pos, show_quadrants=True)
find_safety_factor(pos)

224969976

In [94]:
def possible_tree(positions):
    """Heuristic for possible tree based on clustering"""
    mean_x = sum(p[0] for p in positions)
    mean_y = sum(p[1] for p in positions)
    return sum((p[0]-mean_x)**2 + (p[1]-mean_y)**2 for p in positions)

def animate(parsed_data) -> None:
    max_time = 10000
    sleep = 5
    collect = []
    for t in range(max_time):
        positions = find_positions(parsed_data, t)
        h = possible_tree(positions)
        header = 'time: '+str(t)+' heuristic:'+str(h)
        s = print_positions(positions)
        collect.append((h, header, s))
    collect.sort()
    for c in collect:
        _, header, s = c
        clear_output(wait=True)
        print(header)
        print(s)
        time.sleep(sleep)
        
def print_positions_after(parsed_data, t):
    positions = find_positions(parsed_data, t)
    heuristic = possible_tree(positions)
    print('time:', t, 'heuristic:',heuristic ) 
    print(print_positions(positions))
    
animate(parsed_data)
# print_positions_after(parsed_data, 7892)

time: 3852 heuristic:487699749837
..................................1.1.3....11.......1.............1..................................
1.............1........1...........2..12...1.........................................................
.......................3............1...1..1......................1..................................
..............1...............1......11.......11.....................................................
.......................1..............1..11..........................................................
............................12....11...1....1.1......................................................
........................1......1....1..........1..........................................1..........
.....................1..............1...1..1.........................................................
.......................2...1....1........2...........................................................
...............................................1

KeyboardInterrupt: 